# INSTALLING AND IMPORTING DEPENDENCIES

In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.


  Using cached mediapipe-0.10.5-cp310-cp310-win_amd64.whl (50.3 MB)
     ------------------------------------ 904.0/904.0 kB 346.7 kB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.21.0
    Uninstalling protobuf-4.21.0:
      Successfully uninstalled protobuf-4.21.0


In [1]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe feed', frame)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

# MAKING DETECTIONS

In [16]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                )
        
        cv2.imshow('Mediapipe Feed', image)
#         print(results)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [17]:
print(results.pose_landmarks)


landmark {
  x: 0.6232461929321289
  y: 0.5151922106742859
  z: -0.9419460296630859
  visibility: 0.9999113082885742
}
landmark {
  x: 0.6527971029281616
  y: 0.4449119567871094
  z: -0.8634448051452637
  visibility: 0.9998098611831665
}
landmark {
  x: 0.6711825132369995
  y: 0.4481302797794342
  z: -0.8632872700691223
  visibility: 0.9997947812080383
}
landmark {
  x: 0.6896390914916992
  y: 0.4518274664878845
  z: -0.8628061413764954
  visibility: 0.9997498393058777
}
landmark {
  x: 0.5891015529632568
  y: 0.4385654628276825
  z: -0.8732302784919739
  visibility: 0.9998499751091003
}
landmark {
  x: 0.5645679235458374
  y: 0.4383821487426758
  z: -0.8721941113471985
  visibility: 0.999858558177948
}
landmark {
  x: 0.5379396080970764
  y: 0.44046980142593384
  z: -0.8724095225334167
  visibility: 0.9998671412467957
}
landmark {
  x: 0.7107423543930054
  y: 0.49230676889419556
  z: -0.4274265766143799
  visibility: 0.9996755719184875
}
landmark {
  x: 0.49821239709854126
  y: 0.4797

# DETERMINING JOINTS

In [2]:
cap = cv2.VideoCapture(0)
## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                )
        
        cv2.imshow('Mediapipe Feed', image)
#         print(results)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# CALCULATING ANGLE AND COUNTING CURLS
<img src = "https://mediapipe.dev/images/mobile/pose_tracking_full_body_landmarks.png" style = "height = 300px" >-


In [4]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360 - angle
    
    return angle

In [19]:
def get_coordinates(motion):
    if motion == "left bicep curl":
        l_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        l_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        l_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        
        
        angle = calculate_angle(l_shoulder, l_elbow, l_wrist)
        
        return angle
    if motion == "right bicep curl":
        r_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
        r_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
        r_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
        
        angle = calculate_angle(r_shoulder, r_elbow, r_wrist)
        
        return angle

In [14]:
print(get_coordinates("left bicep curl"))

163.55546589249735


In [20]:
cap = cv2.VideoCapture(0)

stage = None
counter = 0

exercise = input("Enter exercise - ")

angle_specified_down = int(input("Enter the angle for stage 'down'- "))
angle_specified_up = int(input("Enter the angle for stage 'up'- "))

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        try:
            landmarks = results.pose_landmarks.landmark
        
            angle = get_coordinates(exercise)
            
            cv2.putText(image, str(angle), 
                           tuple(np.multiply(elbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
            
            #curl counter logic            
            if angle > angle_specified_down:
                stage = "down"
            if angle < angle_specified_up and stage =='down':
                stage="up"
                counter +=1
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                )
        
        cv2.imshow('Mediapipe Feed', image)
#         print(results)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

Enter exercise - right bicep curl
Enter the angle for stage 'down'- 160
Enter the angle for stage 'up'- 90
